In [ ]:
import sqlalchemy

In [ ]:
sqlalchemy.create_engine('sqlite:///vivino_v2.db')

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///vivino_v2.db

We want to highlight 10 wines to increase our sales, which ones should we choose and why?

In [ ]:
%%sql -- DONE
SELECT
    name,
    ratings_average AS rating, ratings_count
FROM
    wines
ORDER BY
    ratings_count DESC,rating DESC
LIMIT
    10

We have a marketing budget for this year, which country should we prioritise and why?

In [ ]:
%%sql --DONE
SELECT
    *
FROM
    countries
ORDER BY
    users_count DESC
    
LIMIT
    3

We would like to give a price to the best winery, which one should we choose and why?

In [ ]:
%%sql --Complete
SELECT wineries.id, wineries.name, wines.ratings_average, SUM(wines.ratings_count)
FROM wineries
JOIN wines
ON wines.winery_id = wineries.id
GROUP BY wineries.name
ORDER BY wines.ratings_count DESC
LIMIT 3

We has detected that a big cluster of customers like a specific combination of tastes. We have identified few primary keywords that matches this and we would like you to find all the wines that have those keywords. To ensure accuracy of our selection, ensure that more than 10 users confirmed those keywords. Also, identify the flavor_groups related to those keywords.

In [ ]:
%%sql--TEST
SELECT wines.id, keywords.name as keyword, wines.name as wine, keywords_wine.keyword_type
FROM wines
JOIN keywords_wine
ON wines.id = keywords_wine.wine_id
JOIN keywords
ON keywords.id = keywords_wine.keyword_id
WHERE keywords.name IN ('coffee','toast','green apple', 'cream', 'citrus') AND (keywords_wine.count > 10) AND keywords_wine.keyword_type = 'primary'
LIMIT 5
--117,417,195,123,106

In [ ]:
%%sql
SELECT wines.name AS wine,wines.id as wine_id, keywords_wine.group_name
FROM wines
JOIN keywords_wine ON wines.id = keywords_wine.wine_id
JOIN keywords ON keywords.id = keywords_wine.keyword_id
WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
AND keywords_wine.keyword_type = 'primary'
GROUP BY wines.id, wines.name
HAVING COUNT(keywords.id) = 5
AND keywords_wine.count > 10

In [ ]:
%%sql--TESTS
SELECT wines.name AS wine,wines.id as wine_id, COUNT(keywords.id) AS keyword_count, keywords.name
FROM wines
JOIN keywords_wine ON wines.id = keywords_wine.wine_id
JOIN keywords ON keywords.id = keywords_wine.keyword_id
WHERE keywords.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
  AND keywords_wine.keyword_type = 'primary'
GROUP BY wines.id, wines.name
HAVING COUNT(keywords.id) = 5
AND keywords_wine.count > 10
   

In [ ]:
%%sql -- TESTS
SELECT  keywords.name as keyword, wines.name as wine
FROM keywords
JOIN wines
ON wines.id = keywords_wine.wine_id
JOIN keywords
ON keywords.id = keywords_wine.keyword_id
--WHERE keywords.name IN ('coffee','toast','green apple', 'cream', 'citrus') AND (keywords_wine.count > 10)
--Adding a lot of ands, so wines have ALL the keywords
--WHERE (keywords.name = 'coffee') AND (keywords.name = 'toast') AND (keywords.name = 'green apple') AND (keywords.name = 'cream') AND (keywords.name = 'citrus')

We would like to do a selection of wines that are easy to find all over the world. Find the top 3 most common grape all over the world and for each grape, give us the the 5 best rated wines.

In [ ]:
%%sql --COMPLETE
SELECT grapes.name AS grape_name, grapes.id AS grape_id, SUM(most_used_grapes_per_country.wines_count) AS total_world_wide
FROM most_used_grapes_per_country
JOIN grapes
ON grapes.id = most_used_grapes_per_country.grape_id
GROUP BY most_used_grapes_per_country.wines_count
ORDER BY most_used_grapes_per_country.wines_count DESC
LIMIT 3

We would to give create a country leaderboard, give us a visual that shows the average wine rating for each country. Do the same for the vintages.

In [ ]:
%%sql -- WITHOUT VINTAGES
SELECT DISTINCT countries.name as country, ROUND(AVG(wines.ratings_average),2) AS average_rating
from countries
JOIN regions
ON countries.code = regions.country_code
JOIN wines
ON wines.region_id = regions.id
GROUP BY countries.name
ORDER BY average_rating DESC


In [ ]:
%%sql -- INCLUDED VINTAGES
SELECT DISTINCT countries.name as country, ROUND(AVG(vintages.ratings_average),2) AS average_ratings_vintage
from countries
JOIN regions
ON countries.code = regions.country_code
JOIN wines
ON wines.region_id = regions.id
-- adding vintages
JOIN vintages
ON vintages.wine_id = wines.id
GROUP BY countries.name
ORDER BY average_ratings_vintage DESC


Give us any other useful insights you found in our data. Be creative!

**Nice-to-have features**

One of our VIP client like Cabernet Sauvignon, he would like a top 5 recommandation, which wines would you recommend to him?

In [ ]:
%%sql--WITH GRAPE
SELECT wines.id AS wine_id, wines.name
FROM wines
JOIN keywords_wine
ON keywords_wine.wine_id = wines.id
JOIN keywords
ON keywords_wine.keyword_id = keywords.id
WHERE keywords_wine.keyword_id = 2
LIMIT 5